## Preprocessing

In [1]:
# STEP 1
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback




# STEP 2
#  Import and read the charity_data.csv.

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# STEP 3
# # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# STEP 4
# # Determine the number of unique values in each column.
unique_values = application_df.nunique()

In [4]:
# STEP 5
# # Display the number of unique values for each column
print("\nNumber of Unique Values per Column:")
print(unique_values)



Number of Unique Values per Column:
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
#For columns with more than 10 unique values,
#determine the number of data points for each unique value and print them.

# Loop through each column and find columns with more than 10 unique values
for column in application_df.columns:
    if application_df[column].nunique() > 10:
        print(f"\nColumn: {column} (Unique Values: {application_df[column].nunique()})")

        # Count the number of data points for each unique value in the column
        value_counts = application_df[column].value_counts()

        # Print the value counts for the column
        print(value_counts)


Column: APPLICATION_TYPE (Unique Values: 17)
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

Column: CLASSIFICATION (Unique Values: 71)
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

Column: ASK_AMT (Unique Values: 8747)
ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64


In [6]:
# STEP 6
# # Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Use the `value_counts()` method to count the occurrences of each unique value in the 'APPLICATION_TYPE' column
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# STEP 7
# # Display the value counts for 'APPLICATION_TYPE'
print("\nValue Counts for 'APPLICATION_TYPE':")
print(application_type_counts)


Value Counts for 'APPLICATION_TYPE':
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [7]:
# STEP 8
# # Choose a cutoff value of 500 and create a list of application types to be replaced
# This defines a threshold to determine which 'APPLICATION_TYPE' categories are considered rare.
# Categories with counts below this threshold will be replaced with 'Other'.
# This reduces the number of categories, simplifying the dataset, and improving model performance.
# use the variable name `application_types_to_replace`

# Define the cutoff value
threshold = 500

# Create a list of application types that have fewer than 'threshold' occurrences of 500
# - application_type_counts < threshold: Creates a boolean Series where True indicates counts below the threshold
# - [application_type_counts < threshold]: Filters the Series to include only those rows where the condition is True
# - .index.tolist(): Extracts the index (application type names) of the filtered Series and converts it to a list
application_types_to_replace = application_type_counts[application_type_counts < threshold].index.tolist()


# Display the list of application types to be replaced with 'Other'
print("\nApplication Types to be Replaced with 'Other':")
print(application_types_to_replace)


Application Types to be Replaced with 'Other':
['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


In [8]:
# STEP 9
# Replace all occurrences of rare application types (identified in 'application_types_to_replace') with the label 'Other'.
# This consolidates infrequent categories into a single category, reducing noise and simplifying the dataset.
# - 'APPLICATION_TYPE' is the column modified
# - 'application_types_to_replace' is the list of values to be replaced
# - 'Other' is the new value that replaces the rare categories

In [9]:
# STEP 10
# # Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
# Verify the changes by displaying the updated 'APPLICATION_TYPE' value counts
print("\nUpdated 'APPLICATION_TYPE' Value Counts After Replacement:")
print(application_df['APPLICATION_TYPE'].value_counts())


Updated 'APPLICATION_TYPE' Value Counts After Replacement:
APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [10]:
# Step 11: Examine the 'CLASSIFICATION' Value Counts

# Use the `value_counts()` method to count the occurrences of each unique value in the 'CLASSIFICATION' column
classification_counts = application_df['CLASSIFICATION'].value_counts()
# Display the value counts for 'CLASSIFICATION'
print("\nValue Counts for 'CLASSIFICATION':")
print(classification_counts)


Value Counts for 'CLASSIFICATION':
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [11]:
#step 12
#  Look at CLASSIFICATION value counts > 1
#  Identify classifications that have more than 1 occurrence
# Filter the classifications to include only those with more than one occurrence,
# which identifies meaningful classes for the model and handles rare classes appropriately.
classification_counts_gt1 = classification_counts[classification_counts > 1]

# Display the classifications with counts greater than 1
print("\n'CLASSIFICATION' Categories with More Than 1 Occurrence:")
print(classification_counts_gt1)


'CLASSIFICATION' Categories with More Than 1 Occurrence:
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [12]:
# Step 13:
#  Choose a cutoff value of 1000 and create a list of classifications to be replaced
# This list replaces less frequent classifications with a common category, called "Other".
# This helps in reducing the complexity of the model by minimizing the number of unique classes.

# Define the cutoff value
cutoff = 1000
# Any classification with fewer than 1000 occurrences will be replaced

#classifications_to_replace = classification_counts_gt1[classification_counts_gt1 < cutoff].index.tolist()
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Display the list of classifications that will be replaced with 'Other'
print("\nClassifications to be Replaced with 'Other':")
print(classifications_to_replace)



Classifications to be Replaced with 'Other':
['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


In [13]:
#Step 14
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [14]:
# Step 15: Convert categorical data to numeric with `pd.get_dummies`

# Use pd.get_dummies() to convert categorical columns into numeric columns
# This function automatically handles categorical columns by converting them to a binary (0 or 1) format

application_df_encoded = pd.get_dummies(application_df)

# Display the first few rows of the newly encoded dataframe
# This will show how the categorical columns have been transformed into numerical ones
print("\nEncoded DataFrame (First 5 Rows):")
print(application_df_encoded.head())



Encoded DataFrame (First 5 Rows):
   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                   False   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                  True                 False                False   
1                 False                 False                 True   
2                 False                 False                False   
3                 False                 False                 True   
4                 False                 False                 True   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                False                False                False  ...   
1                False         

In [15]:
# Step 16: Split the Data into Features (X) and Target (y)

# The target variable is 'IS_SUCCESSFUL' is the target column.
#
y = application_df_encoded['IS_SUCCESSFUL'].values

# The features will include all other columns except for the target
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL']).values

# Split the Data into Training and Testing Sets

# Use train_test_split to split the data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing,
# and 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify the split
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)


Training Features Shape: (27439, 43)
Testing Features Shape: (6860, 43)
Training Target Shape: (27439,)
Testing Target Shape: (6860,)


In [16]:
#Step 17:   Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Step 18: Create the Neural Network Model Using TensorFlow and Keras

# Step 18.1: Determine the Number of Input Features
# The number of input features is equal to the number of columns in X_train_scaled
num_features = X_train_scaled.shape[1]
print(f"Number of input features: {num_features}")



Number of input features: 43


In [18]:

# Step 18.2: Initialize the Sequential Model
# The Sequential API buildsthe model layer by layer
nn = tf.keras.models.Sequential()

# Step 18.3: Add the First Hidden Layer
# - Number of neurons in the first layer

first_neuron = 400

# - Activation function: ReLU (Rectified Linear Unit) is commonly used for hidden layers
# - Input dim: Specifies the shape of the input data (only needed for the first layer)
nn.add(layers.Dense(first_neuron, activation='relu', input_dim=num_features))

# Step 18.4: Add the Second Hidden Layer
# Adding another hidden layer helps the model learn more complex patterns
# - Number of neurons neurons in the second hidden layer.
# - Using ReLU activation function.
# - No need to specify `input_shape` for subsequent layers as Keras automatically infers it.

second_neuron = 200

# - Activation function: ReLU
nn.add(layers.Dense(second_neuron, activation='relu'))

# Step 18.5: Add the Output Layer
# - For binary classification, use 1 neuron with a sigmoid activation function
# - Activation function: Sigmoid to output a probability between 0 and 1
# - `1` neuron because we're predicting a single binary outcome.
# - `activation='sigmoid'` outputs a value between 0 and 1, representing the probability of the positive class.

nn.add(layers.Dense(1, activation='sigmoid'))

# Step 18.6: Review the Model Structure
# Print a summary of the model architecture, including layers, output shapes, and the number of parameters
# Check the structure of the model  - `model.summary()` details an overview of the model, verifying the architecture.

nn.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 400)                 │          17,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 98,001 (382.82 KB)

 Trainable params: 98,001 (382.82 KB)

 Non-trainable params: 0 (0.00 B)

Explanation of the Summary:

*   Layer Names: Each layer is named automatically (e.g., dense, dense_1, dense_2)
*   Output Shapes: Shows the shape of the output from each layer.(including neurons)

*   Param #: Indicates the number of parameters (weights and biases) in each layer.
*   Total Params: Total number of trainable parameters in the model.



In [19]:
# Step 19.1: Compile the Neural Network Model

# Compile the model with the following configurations:
# - Optimizer: Adam optimizer is efficient & adjusts the learning rate during training for convergence.
# - Loss Function: Binary Crossentropy is suitable for binary classification tasks.
#                  It measures the performance of the model where the output is a probability value between 0 and 1.
# - Metrics: Monitor 'accuracy' to evaluate the performance of the model.
#                  It tracks the accuracy of the model during training and evaluation.

nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])


In [20]:
# Step 19.2: Create a  Callback to Save Weights Every Five Epochs
class SaveEveryNEpochs(Callback):
    def __init__(self, save_freq, filepath):
        super(SaveEveryNEpochs, self).__init__()
        self.save_freq = save_freq
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        # Save the model every 'save_freq' epochs
        if (epoch + 1) % self.save_freq == 0:
            self.model.save_weights(self.filepath.format(epoch=epoch + 1))
            print(f'\nSaved weights at epoch {epoch + 1}')

# Initialize the custom callback to save weights every five epochs
checkpoint_callback = SaveEveryNEpochs(
    save_freq=5,
    filepath='weights_epoch_{epoch:02d}.weights.h5'
)


In [21]:
# Step 19.3: Train the Neural Network Model

# Define the number of epochs (iterations over the entire dataset)
epochs = 600

# Define the batch size (number of samples per gradient update)
batch_size = 200

# Train the model using the training data
# - X_train_scaled: Preprocessed and scaled training features
# - y_train: Training target variable
# - epochs: Number of times the entire training dataset is passed through the model
# - batch_size: Number of samples per gradient update
# - validation_data: Tuple (X_test_scaled, y_test) to evaluate the loss and any model metrics at the end of each epoch
# - callbacks: List of callback instances to apply during training (e.g., checkpoint_callback)

history = nn.fit(X_train_scaled, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_test_scaled, y_test),
                callbacks=[checkpoint_callback],
                verbose=1)

# Explanation:
# - `epochs=600`: The model will iterate 600 times over the entire training dataset.
# - `batch_size=200`: The model updates its weights after every 200 samples.
# - `validation_data=(X_test_scaled, y_test)`: The model evaluates performance on the testing data at the end of each epoch.
# - `callbacks=[checkpoint_callback]`: Incorporates the custom callback to save model weights every five epochs.
# - `verbose=1`: Displays a progress bar and training metrics during training.

Epoch 1/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.7087 - loss: 0.5912 - val_accuracy: 0.7195 - val_loss: 0.5669
Epoch 2/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7330 - loss: 0.5526 - val_accuracy: 0.7214 - val_loss: 0.5608
Epoch 3/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7282 - loss: 0.5529 - val_accuracy: 0.7283 - val_loss: 0.5576
Epoch 4/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7301 - loss: 0.5476 - val_accuracy: 0.7252 - val_loss: 0.5598
Epoch 5/600
133/138 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7364 - loss: 0.5451
Saved weights at epoch 5
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7362 - loss: 0.5452 - val_accuracy: 0.7259 - val_loss: 0.5606
Epoch 6/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7320 - loss: 0.5472 - val_accuracy: 0.7243 - val_loss: 0.5589
Epoch 7/600
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7346 - loss: 0.5402 - val_accuracy: 0.7277 - val_los

In [22]:
# Step 19.4: Evaluate the Neural Network Model on Test Data

# Evaluate the trained model using the testing data
# - X_test_scaled: Preprocessed and scaled testing features
# - y_test: Testing target variable

test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=0)

# Print the evaluation results
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Explanation:
# - `nn.evaluate()`: Computes the loss and specified metrics for the model on the provided data.
# - `verbose=0`: Suppresses the progress bar during evaluation.
# - `test_loss`: The loss value on the testing data.
# - `test_accuracy`: The accuracy of the model on the testing data.



Test Loss: 0.6284
Test Accuracy: 0.7255


In [23]:
# Step 19.5: Save the Trained Model to an HDF5 File

from google.colab import drive
drive.mount('/content/drive')

# Define the filename for the saved model
# model_filename = 'AlphabetSoupCharity.h5'
model_filename = '/content/drive/My Drive/deep-learning-challenge/AlphabetSoupCharity-O3.h5'
# Save the entire model to the HDF5 file
nn.save(model_filename)

# Confirmation message
print(f"\nModel has been saved to {model_filename}")




Mounted at /content/drive

Model has been saved to /content/drive/My Drive/deep-learning-challenge/AlphabetSoupCharity-O3.h5
